<b> Lab 4 | Week 2 Day 4 </b> 

---

<b> Deep Research</b>  
--
One of the classic cross-business use cases of Agentic AI! This is huge. 

In this lab, we will implement a Deep Reasearch Agent.  
We will use the following in the orchestration:
- Tools
- Structured Output
- Hosted Tools from OpenAI

<b> OpenAI Hosted Tools </b>

OpenAI Agents SDK includes the following hosted tools:

- The `WebSearchTool` lets an agent search the web.  
- The `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.  
- The `ComputerTool` allows automating computer use tasks like taking screenshots and clicking.

In [ ]:
import os, sendgrid, asyncio
from dotenv import load_dotenv
from agents import Agent, Runner, trace, WebSearchTool,function_tool, gen_trace_id
from agents.model_settings import ModelSettings
from typing import Dict
from sendgrid.helpers.mail import Mail, Email, To, Content
from pydantic import BaseModel, Field
from IPython.display import display, Markdown

In [ ]:
load_dotenv(override=True)

In [ ]:
# Defining the Search Agent

SA_INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must be 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=SA_INSTRUCTIONS,
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
    tools=[WebSearchTool(search_context_size="low")],
)

In [ ]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

As always, take a look at the trace | https://platform.openai.com/traces

***

<b>Now we will build a Planner Agent</b>
  
This agent is responsible to take the input query and come up with a handful of searches that should be run to do the Deep
Reasearch. It will output a list of queries and their respective reasons

Time to:
- Use Structured O/p
- Include a description of the fields

In [ ]:
# Use Pydantic to define the Schema of our response - this is known as "Structured Outputs"
# With massive thanks to student Wes C. for discovering and fixing a nasty bug with this!

class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")
    query: str = Field(description="The search term to use for the web search.")

class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")

# Set the number of searches to perform. Unless limited, this can add upto bigger Web Search costs

SEARCH_COUNT = 3

PA_INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {SEARCH_COUNT} terms to query for."

planner_agent = Agent(
    name="Planner Agent",
    instructions=PA_INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

In [ ]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search Plan"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

In [ ]:
# Setup the Function Tool to send HTML email

@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """

    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get("SENDGRID_API_KEY"))

    from_email = Email("skillshift.ai@gmail.com") # SendGrid verified sender
    to_email = To("melbith@gmail.com")
    content = Content("text/html", html_body)

    mail = Mail(from_email, to_email, subject, content)

    try:
        sg.client.mail.send.post(request_body=mail.get())
        return {"status": "success"}
    except Exception as e:
        return {"status": "error", "message": str(e)}

In [ ]:
send_email

Define the Email Agent

In [ ]:
EA_INSTRUCTIONS = """
You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line.
"""

email_agent = Agent(
    name="Email Agent",
    instructions=EA_INSTRUCTIONS,
    model="gpt-4o-mini",
    tools=[send_email]
)

Define the Writer Agent

In [ ]:
# Pydantic Structure for output
class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")

WA_INSTRUCTIONS = """
You are a senior researcher tasked with writing a cohesive report for a research query.
You will be provided with the original query, and some initial research done by a research assistant.
You should first come up with an outline for the report that describes the structure and
flow of the report. Then, generate the report and return that as your final output.
The final output should be in markdown format, and it should be lengthy and detailed. Aim for 5-10 pages of content, 
at least 1000 words."
"""

writer_agent = Agent(
    name="Writer Agent",
    instructions=WA_INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

Following three Co-routines (async functions) will plan and execute the search using the Planner Agent and the Search Agent

In [ ]:
# 1) This function calls the Planner Agent and pass on the query and receives the list of Web search Items in return

async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning Searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    
    # Return the WebSearchPlan with the list of Web search Items 
    return result.final_output

# 2) This function is responsible to collect the Web search Items and Initiate the searches by creating async tasks
# for the individual search items and run those tasks concurrently by calling the perform_search() function below

async def organize_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Initiating search...")
    tasks = [asyncio.create_task(perform_search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks) # Run co-routines concurrently (not in parallel). *tasks unpack tasks
    print("Finished searching")
    return results

# 3) This is the function that actually executes the search task created by the above function
# This invokes the Search Agent for each item in the search plan

async def perform_search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search Item: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output


Next two Co-routines will do the housekeeping activities: 
- Wirte the Final Report and 
- Email it

In [ ]:
# 4) This function is responsible to collect the original query and search results and write the final report
# by invoking the Writer Agent. It returns the final HTML report to be sent

async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")

    input = f"Original Query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)

    print("Finished writing report")
    return result.final_output

# 4) This function is responsible to collect the final report produced by the Writer Agent, in the ReportData object
# and send it by email by invoking the Email Agent. 

async def send_email(report: ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return report

### Showtime !

In [ ]:
query = "Latest AI Agent frameworks in 2025"

with trace("Deep Research"):
    print("Starting research...")

    # Call CR #1 to plan the searches
    search_plan = await plan_searches(query)

    # Call CR #2 (which calls CR #3 in turn) to initiate the serach tasks and run them concurrently
    serach_results = await organize_searches(search_plan)

    # Call CR #4 to write the final report
    report = await write_report(query, serach_results)

    # Call CR #5 to send the email with final report
    await send_email(report)

    print("Reaserach Complete!")

As always, take a look at the trace | https://platform.openai.com/traces